In [1]:
from CompactFIPS202 import *
import binascii
import os
from hashlib import sha3_256, sha3_512, shake_128, shake_256
from kyber import Kyber512, Kyber768, Kyber1024
from aes256_ctr_drbg import AES256_CTR_DRBG
import itertools
import utils

os.system('rm -rf ./vec')

def parse_kat_data(data):
    parsed_data = {}
    count_blocks = data.split('\n\n')
    for block in count_blocks[1:-1]:
        block_data = block.split('\n')
        count, seed, pk, sk, ct, ss = [line.split(" = ")[-1] for line in block_data]
        parsed_data[count] = {
            "seed": bytes.fromhex(seed),
            "pk": bytes.fromhex(pk),
            "sk": bytes.fromhex(sk),
            "ct": bytes.fromhex(ct),
            "ss": bytes.fromhex(ss),   
        }
    return parsed_data

In [2]:
with open("assets/PQCkemKAT_1632.rsp") as f:
    kat_data = f.read()
    parsed_data = parse_kat_data(kat_data)

data = parsed_data['0']
seed, pk, sk, ct, ss = data.values()
        
# Seed DRBG with KAT seed
Kyber512.set_drbg_seed(seed)
# Assert keygen matches
_pk, _sk = Kyber512.keygen()

# Assert encapsulation matches
_ct, _ss = Kyber512.enc(_pk)

#    # Assert decapsulation matches
#    __ss = Kyber512.dec(ct, sk)

[CBD] Input Bytes  : b'\x8a\x15-\x07;\x12\x16/Ge\xdf\xf4\xcbe\x8b\xe3c\x17<\xe3\x96\x9c\xf5\xe2\xf4\xe5c\xcc>U\xc6\xd7\xb2\xc2\xe3\n\x90O;\xa4X8\xd8\x96\xf7\x95\xd9\x8e\xdeJ\xa6\x82\xaf\xd2F\x18V<\xe2\x92\xc5?\xb6\xb5\x1a\x88\x19!\x94\xbc\x01\x07\xe6\x92\x1a\xc2c\x04=\xf6\x08\x9f^\x08\xfb\xd5\xcc?\x9d9\x13\xb4\xb0T\xee\xe5x\x05\x12\xe6\xa8l\xda\xeeJ\xcc\xd56\x92"\xad\xe8\x86\xd7\xbfZ\xbb\xea\xf2\xd2Cr1\xb2H\x85{\x9b\xa5\xcd&\x89J\x1bx40vV\x1a\xa4\xdc\x1f\xf2\xde\xc7s\xcf\xd0s\x7f$\x8f\xd8.T\xfd\x90\xeb\xc2\xe3\xfb\xe5d\x86\xe5b\x88\xbe_\x15\xbeQ\xa5\x13M\x8c\xbf\xe4\xb3;\xac\xa2\xb4\x1d\x9a+\xf7\x95\xf9^\x05'
[CBD] ETA          : 3
[CBD] COEFF        : [0, 1, 0, 1, 3, -1, 1, 1, 1, -2, -1, 0, 1, -1, 1, -1, 1, 2, 0, 0, 1, 2, 0, 2, -2, 0, 0, 1, -1, 1, 1, -3, -1, 2, -1, -2, 0, -1, 1, 1, 2, 1, 0, -1, 1, 2, 0, 1, -1, -1, -1, 1, 0, 0, 2, 1, -2, 0, -2, -1, 1, 1, 0, 1, 0, 0, 0, -1, 1, -1, -1, -1, 2, 0, 0, 1, -2, -1, 0, 0, 2, 0, 1, 0, 0, -1, 1, 0, -1, 1, 1, 1, -2, 2, -2, 1, 1, 1, -1, 2, 1, 2, 2,

In [3]:
(pk == _pk) == (sk == _sk) == (ct == _ct) == (ss == _ss)

True

In [4]:
Kyber512.R.cbd_input_bytes.hex()

'6d98e3a6687a1ea0f8e7e20b0bc69274afd5c52b2ab946bba8424252930a98790cb72b7d701a18b76ce223d2bba02ebb6fcae482f90005822f75a6bda25d46a7a7fdabc048540aaf5930d25669487d688ba4b59ba0b4d5e0c04ba9d9575581461145e6f6957eb97dbf4d30f9e07fee153db369bee1bcf52f9f41e9297a151c36'

In [5]:
format(Kyber512.R.cbd_input_bytes[0], '08b')[::-1]

'10110110'

In [6]:
Kyber512.R.cbd_input_bytes.hex()

'6d98e3a6687a1ea0f8e7e20b0bc69274afd5c52b2ab946bba8424252930a98790cb72b7d701a18b76ce223d2bba02ebb6fcae482f90005822f75a6bda25d46a7a7fdabc048540aaf5930d25669487d688ba4b59ba0b4d5e0c04ba9d9575581461145e6f6957eb97dbf4d30f9e07fee153db369bee1bcf52f9f41e9297a151c36'

In [7]:
i = 2
Kyber512.R.cbd_list_of_bits[0+8*i:8+8*i]

[1, 1, 0, 0, 0, 1, 1, 1]